In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
import time
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import os
import gym_Aircraft
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
env = gym.make("acav-v0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/hyogun/anaconda3/envs/collision_avoidance/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class FClayer(nn.Module):
    def __init__(self, innodes: int, nodes: int):
        super(FClayer, self).__init__()
        self.fc=nn.Linear(innodes,nodes)
        self.act=nn.LeakyReLU(0.2, inplace=True)
    def forward(self, x: Tensor) -> Tensor:
        out=self.fc(x)
        out=self.act(out)
        return out
    
class WaveNET(nn.Module):
    def __init__(self, block: Type[Union[FClayer]], planes: List[int], nodes: List[int], num_classes: int = 3
                ) -> None:
        super(WaveNET, self).__init__()
        self.innodes=5
        
        self.layer1=self._make_layer(block, planes[0], nodes[0])
        self.layer2=self._make_layer(block, planes[1], nodes[1])
        self.layer3=self._make_layer(block, planes[2], nodes[2])
        
        self.fin_fc=nn.Linear(self.innodes,num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='leaky_relu')
    
    def _make_layer(self, block: Type[Union[FClayer]], planes: int, nodes: int) -> nn.Sequential:

        layers = []
        layers.append(block(self.innodes, nodes))
        self.innodes = nodes
        for _ in range(1, planes):
            layers.append(block(self.innodes, nodes))

        return nn.Sequential(*layers)

        
    def _forward_impl(self, x: Tensor) -> Tensor:
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.fin_fc(x)
        
        return x
    
    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [5]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

In [6]:
policy_net = torch.load("./Custom_model_fin").to(device)
target_net = torch.load("./Custom_model_fin").to(device)
target_net.eval()

/home/hyogun/anaconda3/envs/collision_avoidance/lib/python3.8/site-packages/torch/serialization.py:624: UserWarning: Couldn't retrieve source code for container of type WaveNET. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container of "
/home/hyogun/anaconda3/envs/collision_avoidance/lib/python3.8/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/hyogun/anaconda3/envs/collision_avoidance/lib/python3.8/site-packages/torch/serialization.py:624: UserWarning: Couldn't retrieve source code for container of type FClayer. It won't be checked for correctness upon loading.
  warnings.warn("Couldn't retrieve source code for container 

WaveNET(
  (layer1): Sequential(
    (0): FClayer(
      (fc): Linear(in_features=5, out_features=40, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): FClayer(
      (fc): Linear(in_features=40, out_features=40, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (layer2): Sequential(
    (0): FClayer(
      (fc): Linear(in_features=40, out_features=20, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): FClayer(
      (fc): Linear(in_features=20, out_features=20, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (layer3): Sequential(
    (0): FClayer(
      (fc): Linear(in_features=20, out_features=60, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (1): FClayer(
      (fc): Linear(in_features=60, out_features=60, bias=True)
      (act): LeakyReLU(negative_slope=0.2, inplace=True)
    )
  )
  (fin_fc): Linear(in_features=60, out_featu

In [7]:
n_actions = env.action_space.n

In [8]:
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(50000)

In [9]:
steps_done = 0

In [10]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [11]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # 기대 Q 값 계산
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Huber 손실 계산
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 50000
total_res=[]
reward_list=[]
for i_episode in range(num_episodes):
    total_reward=0
    
    # 환경과 상태 초기화
    res_list=np.zeros(11)
    state = env.reset()
    state=torch.from_numpy(state.astype(np.float32)).unsqueeze(0).to(device)
    for t in count():
        # 행동 선택과 수행
        
        action = select_action(state)
        next_state, reward, done, res = env.step(action.item())
        reward = torch.tensor([reward], dtype=torch.float32).to(device)
        
        next_state=torch.from_numpy(next_state.astype(np.float32)).unsqueeze(0).to(device)

        # 새로운 상태 관찰
        if not done:
            next_state = next_state
        else:
            next_state = None

        # 메모리에 변이 저장
        memory.push(state, action, next_state, reward)

        # 다음 상태로 이동
        state = next_state

        # 최적화 한단계 수행(목표 네트워크에서)
        optimize_model()
        
        # Data save
        
        cmd_list,r_list,elev_list,azim_list,Pm_list,Pt_list,h_list=res
        Pm_list=Pm_list.tolist()
        Pt_list=Pt_list.tolist()
        merged_data=itertools.chain([cmd_list],[r_list],[elev_list],[azim_list],Pm_list,Pt_list,[h_list])
        merged_data=np.array(list(merged_data))
        res_list=np.vstack([res_list,merged_data])
        
        total_reward+=reward
        
        if done:
            res_list=np.delete(res_list,0,0)
            
            total_res.append(res_list)
            reward_list.append(total_reward)
            
            now = time.localtime()
#             print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))
            print(f"episode: {i_episode:4d} | final step: {t:3d} | reward: {total_reward.item():4.1f} "
                  f"| sum(R) last 100: {sum(reward_list[-100:]).cpu().numpy()[0]}")
            break
            
        
    #목표 네트워크 업데이트, 모든 웨이트와 바이어스 복사
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.close()

episode:    0 | final step:  98 | reward: -1.0 | E[R] last 100: -1.0
episode:    1 | final step:  98 | reward: -1.0 | E[R] last 100: -2.0
episode:    2 | final step: 226 | reward: 49.6 | E[R] last 100: 47.64065933227539
episode:    3 | final step:  98 | reward: -1.0 | E[R] last 100: 46.64065933227539
episode:    4 | final step:  98 | reward: -1.0 | E[R] last 100: 45.64065933227539
episode:    5 | final step:  98 | reward: -1.0 | E[R] last 100: 44.64065933227539
episode:    6 | final step:  98 | reward: -1.0 | E[R] last 100: 43.64065933227539
episode:    7 | final step:  98 | reward: -1.0 | E[R] last 100: 42.64065933227539
episode:    8 | final step:  98 | reward: -1.0 | E[R] last 100: 41.64065933227539
episode:    9 | final step:  98 | reward: -1.0 | E[R] last 100: 40.64065933227539
episode:   10 | final step:  98 | reward: -1.0 | E[R] last 100: 39.64065933227539
episode:   11 | final step:  98 | reward: -1.0 | E[R] last 100: 38.64065933227539
episode:   12 | final step:  98 | reward: 

episode:  100 | final step:  98 | reward: -1.0 | E[R] last 100: 456.2474365234375
episode:  101 | final step:  98 | reward: -1.0 | E[R] last 100: 456.2474365234375
episode:  102 | final step:  98 | reward: -1.0 | E[R] last 100: 405.6067810058594
episode:  103 | final step:  98 | reward: -1.0 | E[R] last 100: 405.6067810058594
episode:  104 | final step:  99 | reward: -1.0 | E[R] last 100: 405.6067810058594
episode:  105 | final step:  98 | reward: -1.0 | E[R] last 100: 405.6067810058594
episode:  106 | final step:  98 | reward: -1.0 | E[R] last 100: 405.6067810058594
episode:  107 | final step:  99 | reward: -1.0 | E[R] last 100: 405.6067810058594
episode:  108 | final step: 233 | reward: 49.7 | E[R] last 100: 456.2856750488281
episode:  109 | final step:  98 | reward: -1.0 | E[R] last 100: 456.2856750488281
episode:  110 | final step:  98 | reward: -1.0 | E[R] last 100: 456.2856750488281
episode:  111 | final step:  98 | reward: -1.0 | E[R] last 100: 456.2856750488281
episode:  112 | 

episode:  200 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  201 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  202 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  203 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  204 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  205 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  206 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  207 | final step:  98 | reward: -1.0 | E[R] last 100: 102.5552978515625
episode:  208 | final step:  98 | reward: -1.0 | E[R] last 100: 51.876380920410156
episode:  209 | final step:  98 | reward: -1.0 | E[R] last 100: 51.876380920410156
episode:  210 | final step:  98 | reward: -1.0 | E[R] last 100: 51.876380920410156
episode:  211 | final step:  98 | reward: -1.0 | E[R] last 100: 51.876380920410156
episode:  21

episode:  304 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  305 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  306 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  307 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  308 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  309 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  310 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  311 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  312 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  313 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  314 | final step:  98 | reward: -1.0 | E[R] last 100: -49.420841217041016
episode:  315 | final step:  98 | reward: -1.0 | E[R] last 100: -49.42084121

episode:  407 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  408 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  409 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  410 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  411 | final step:  99 | reward: -1.0 | E[R] last 100: -100.0
episode:  412 | final step:  99 | reward: -1.0 | E[R] last 100: -100.0
episode:  413 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  414 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  415 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  416 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  417 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  418 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  419 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episode:  420 | final step:  98 | reward: -1.0 | E[R] last 100: -100.0
episod

...

episode: 17100 | final step:  98 | reward: -1.0 | E[R] last 100: 1.1845779418945312
episode: 17101 | final step: 236 | reward: 49.7 | E[R] last 100: 51.84451675415039
episode: 17102 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17103 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17104 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17105 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17106 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17107 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17108 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17109 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17110 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
episode: 17111 | final step:  98 | reward: -1.0 | E[R] last 100: 51.84451675415039
epi

episode: 17198 | final step:  98 | reward: -1.0 | E[R] last 100: 1.4297599792480469
episode: 17199 | final step:  99 | reward: -1.0 | E[R] last 100: 1.4297599792480469
episode: 17200 | final step:  98 | reward: -1.0 | E[R] last 100: 1.4297599792480469
episode: 17201 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17202 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17203 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17204 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17205 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17206 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17207 | final step:  99 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17208 | final step:  98 | reward: -1.0 | E[R] last 100: -49.23017883300781
episode: 17209 | final step:  98 | reward: -1.0 | E[R] last 100: -49.2301788

episode: 17296 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17297 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17298 | final step:  99 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17299 | final step:  99 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17300 | final step:  99 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17301 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17302 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17303 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17304 | final step:  99 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17305 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17306 | final step:  98 | reward: -1.0 | E[R] last 100: -49.45003890991211
episode: 17307 | final step:  98 | reward: -1.0 | E[R] last 100: -49.4500389

episode: 17394 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17395 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17396 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17397 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17398 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17399 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17400 | final step:  99 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17401 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17402 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17403 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17404 | final step:  98 | reward: -1.0 | E[R] last 100: -49.41315841674805
episode: 17405 | final step:  98 | reward: -1.0 | E[R] last 100: -49.4131584

episode: 17491 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17492 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17493 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17494 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17495 | final step:  99 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17496 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17497 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625
episode: 17498 | final step:  98 | reward: -1.0 | E[R] last 100: -49.346099853515625


In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Episode")
plt.ylabel("Total rewards")
plt.plot(reward_list)

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[9999]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()